### In this file I want to see where the lowest energy gaps lie in the (A_log(s), A_anc(s)) plane. I want to see if passing through the areas of smallest gap gives lowest probability of success for our base run

In [ ]:
import project_lib as mylib
import numpy as np
import matplotlib.pyplot as plt
import glob
import matplotlib.colors
import matplotlib.cm
from mpl_toolkits.axes_grid1 import make_axes_locatable
%matplotlib notebook

In [ ]:
FILES =  glob.glob(r"C:\Users\User\Documents\FIZZIX\4th Year\Project\Data\explore_from_base_QUBITS=4.pkl")
ENDINGS = [f[f.index('Data')+len('Data')+1:][:-4] for f in FILES]

qubits = [end[-1] for end in ENDINGS]
an = [mylib.load_object(end) for end in ENDINGS]

print an[0]['comments']

plt.figure()
plt.title('Non-Neutral hamiltonian Comparison of qubits')
plt.ylabel(r'$P(finding\; ground\; state)$')
plt.xlabel(r'$\theta$ in units $\pi / 2} $')
for i,a in enumerate(an):
    thetas = np.asarray(a['thetas'])
    last_prob = np.asarray(a['last_prob'])
    # here we plot the figure
    plt.plot(2*thetas/np.pi,last_prob, label = qubits[i])
plt.legend(loc = 'best', framealpha= 0)
plt.show()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import os.path
import scipy.linalg # this line is needed as well as the one below so that this imports in Jupyter correctly
import scipy as sp
from tabulate import tabulate
import time
import sys
import multiprocessing as mp
import project_lib as mylib

In [ ]:
# THIS ALL NEEDS TO BE DEFINED SO THAT WE CAN GENERATE THE PROBLEM AND BASE HAMILTONIANS

#define number of logical and ancilla qubits
log_qub = 4
anc_qub = log_qub
qubits = log_qub*2

# define the coupling parameters that comply with the conditions above
JN = 0.1
q0 = 50.
Ja = 100.
Jl = Ja
hl = -Ja+q0

# construct h
hl = np.asarray([-Ja+q0]*log_qub) # h for the logical qubits
ha = np.asarray([-Ja*(2*i-log_qub)+q0 for i in range(1,log_qub+1)]) + np.asarray([JN if (log_qub-i)%2 == 1 else -JN for i in range(1,log_qub+1)])# h for the ancilla qubits
h = np.concatenate((hl,ha)) # the full h

# constuct J
J_log_log = Jl*np.triu(np.ones((log_qub,log_qub)), k=1) # the J component for the logical qubit interactions
J_log_anc = Ja*np.ones((log_qub,log_qub)) # interactions between logical and ancilla bits
Jt = np.concatenate((J_log_log,J_log_anc),axis=1) # make top half of J array
Jb = np.zeros(Jt.shape) # make bottom half of J array
J = np.concatenate((Jt,Jb),axis=0) # the full J

sfl = lambda points: mylib.linear_schedule(points)
sfa = lambda points: mylib.linear_schedule(points)
sfB = mylib.linear_schedule
sched_funcs = [[sfl]*log_qub+[sfa]*anc_qub,[sfB]]
a = mylib.Anneal(qubits,[h,J],T=100,points = 1000,sched_funcs = sched_funcs, diff_scheds = True)

In [ ]:
import matplotlib.image as mpimg
%matplotlib inline
img1=mpimg.imread(r"C:\Users\User\Documents\FIZZIX\4th Year\Project\Jupyter Extra Images\Jup010 - Anneal_path_graph.png")
plt.figure(figsize=(6,6)); plt.imshow(img1)
plt.axis('off')
plt.show()

In [ ]:
# ------------------------------------------------------------------------------------------------------------------------------
#      HERE I WANT TO DEFINE A FUNCTION THAT RETURNS THE HAMILTONIAN AT ANY POINT ON A2 SPACE
# ------------------------------------------------------------------------------------------------------------------------------
'''We need this to be specific to the linear schedule. So for each point we will need to calculate the angle of theta and
thus calculate s so we can feed this into B(s). 
I want to be able create a plot that has sqrt(A_log**2+A_anc**2) = 1 since these are the point that my previous work has 
passed through'''
nax = 1000
sp = np.linspace(0, 1, nax)
gaps = np.zeros((sp.size,sp.size))

def H_A2((log,anc),s):
    return sum([a.Hb[i]*A for i,A in enumerate([log]*log_qub+[anc]*anc_qub)])+s*a.Hp

def gap_finder(M):
    d,P,P_in = mylib.Diagonaliser(M)
    return min(d[d!=min(d)])-min(d)


def gap_A2((log,anc)):
    s = 1-(anc**2+log**2)
    if s<0: return 0
    else: return gap_finder(H_A2((log,anc),s))

for x,log in enumerate(sp):
    print 'running %s of 999'%x
    for y,anc in enumerate(sp):
        gaps[x,y] = gap_A2((log,anc))


In [ ]:
mylib.save_object(gaps,'Jupyter013.001')

In [ ]:
gaps = mylib.load_object('Jupyter013.001')

In [ ]:
def plot(data):
    
    x0, x1, y0, y1 = 0, 1, 0,1 # bounds of data 

    fig,im = plt.subplots(figsize=(6,6))
    ax1 = plt.subplot2grid((1,1), (0,0))
    
    plt.ylabel(r'$A_{anc}(s)$')
    plt.xlabel(r'$A_{log}(s)$')

    im = ax1.imshow(data,extent = (x0,x1, y0,y1, ), origin = 'lower',
cmap=plt.get_cmap('nipy_spectral'), norm=matplotlib.colors.LogNorm(vmin = 10**-5 ,vmax=2))#vmin=1600, vmax=5000
    
    #cb = plt.colorbar(im, orientation='vertical', label = 'gap size')
    plt.show()
    
plot(data)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

from matplotlib.transforms import Affine2D
import mpl_toolkits.axisartist.floating_axes as floating_axes
import mpl_toolkits.axisartist.angle_helper as angle_helper
from matplotlib.projections import PolarAxes
from mpl_toolkits.axisartist.grid_finder import MaxNLocator

# define how your plots look:

def setup_axes(fig, rect, theta, radius):

    # PolarAxes.PolarTransform takes radian. However, we want our coordinate
    # system in degree
    tr = Affine2D().scale(np.pi/180., 1.) + PolarAxes.PolarTransform()

    # Find grid values appropriate for the coordinate (degree).
    # The argument is an approximate number of grids.
    grid_locator1 = angle_helper.LocatorD(2)

    # And also use an appropriate formatter:
    tick_formatter1 = angle_helper.FormatterDMS()

    # set up number of ticks for the r-axis
    grid_locator2 = MaxNLocator(4)

    # the extremes are passed to the function
    grid_helper = floating_axes.GridHelperCurveLinear(tr,
                                extremes=(theta[0], theta[1], radius[0], radius[1]),
                                grid_locator1=grid_locator1,
                                grid_locator2=grid_locator2,
                                tick_formatter1=tick_formatter1,
                                tick_formatter2=None,
                                )

    ax1 = floating_axes.FloatingSubplot(fig, rect, grid_helper=grid_helper)
    fig.add_subplot(ax1)

    # adjust axis
    # the axis artist lets you call axis with
    # "bottom", "top", "left", "right"
    ax1.axis["left"].set_axis_direction("bottom")
    ax1.axis["right"].set_axis_direction("top")

    ax1.axis["bottom"].set_visible(False)
    ax1.axis["top"].set_axis_direction("bottom")
    ax1.axis["top"].toggle(ticklabels=True, label=True)
    ax1.axis["top"].major_ticklabels.set_axis_direction("top")
    ax1.axis["top"].label.set_axis_direction("top")

    ax1.axis["left"].label.set_text("R")
    ax1.axis["top"].label.set_text(ur"$\alpha$ [\u00b0]")

    # create a parasite axes
    aux_ax = ax1.get_aux_axes(tr)

    aux_ax.patch = ax1.patch # for aux_ax to have a clip path as in ax
    ax1.patch.zorder=0.9 # but this has a side effect that the patch is
                         # drawn twice, and possibly over some other
                         # artists. So, we decrease the zorder a bit to
                         # prevent this.

    return ax1, aux_ax

#
# call the plot setup to generate 3 subplots
#

fig = plt.figure(1, figsize=(5, 5))
fig.subplots_adjust(wspace=0.3, left=0.05, right=0.95)

ax1, aux_ax1 = setup_axes(fig, 111, theta=[0, 90], radius=[0, 1])

#
# generate the data to plot
#
theta = np.linspace(0,90) # in degrees
radius = np.cos(6.*theta * np.pi/180.0)**2.0

# 
# populate the three subplots with the data
#
aux_ax1.plot(theta, radius, 'r')

plt.show()  

In [ ]:
x0, x1, y0, y1 = 0, 1, 0,1 # bounds of data 

def plot(gaps):
    
    x0, x1, y0, y1 = 0, 1, 0,1 # bounds of data 
    
    fig,im = plt.subplots(figsize=(6,6))
    ax1 = plt.subplot2grid((1,1), (0,0))
    
    plt.ylabel(r'$A_{anc}(s)$')
    plt.xlabel(r'$A_{log}(s)$')
    #im = ax1.imshow(gaps, origin = 'lower', cmap=plt.get_cmap('nipy_spectral'), norm=matplotlib.colors.LogNorm())
    im = ax1.imshow(gaps,extent = (x0,x1, y0,y1, ), origin = 'lower',
cmap=plt.get_cmap('nipy_spectral'), norm=matplotlib.colors.LogNorm(vmin = 10**-5 ,vmax=2))#vmin=1600, vmax=5000
    
    cb = plt.colorbar(im, orientation='vertical', label = 'gap size')
    plt.show()

import numpy as np
import matplotlib.pyplot as plt

from matplotlib.transforms import Affine2D
import mpl_toolkits.axisartist.floating_axes as floating_axes
import mpl_toolkits.axisartist.angle_helper as angle_helper
from matplotlib.projections import PolarAxes
from mpl_toolkits.axisartist.grid_finder import MaxNLocator

# define how your plots look:

def setup_axes(fig, rect, theta, radius):

    # PolarAxes.PolarTransform takes radian. However, we want our coordinate
    # system in degree
    tr = Affine2D().scale(np.pi/180., 1.) + PolarAxes.PolarTransform()

    # Find grid values appropriate for the coordinate (degree).
    # The argument is an approximate number of grids.
    grid_locator1 = angle_helper.LocatorD(2)

    # And also use an appropriate formatter:
    tick_formatter1 = angle_helper.FormatterDMS()

    # set up number of ticks for the r-axis
    grid_locator2 = MaxNLocator(4)

    # the extremes are passed to the function
    grid_helper = floating_axes.GridHelperCurveLinear(tr,
                                extremes=(theta[0], theta[1], radius[0], radius[1]),
                                grid_locator1=grid_locator1,
                                grid_locator2=grid_locator2,
                                tick_formatter1=tick_formatter1,
                                tick_formatter2=None,
                                )

    ax1 = floating_axes.FloatingSubplot(fig, rect, grid_helper=grid_helper)
    fig.add_subplot(ax1)

    # adjust axis
    # the axis artist lets you call axis with
    # "bottom", "top", "left", "right"
    ax1.axis["left"].set_axis_direction("bottom")
    ax1.axis["right"].set_axis_direction("top")

    ax1.axis["bottom"].set_visible(False)
    ax1.axis["top"].set_axis_direction("bottom")
    ax1.axis["top"].toggle(ticklabels=True, label=True)
    ax1.axis["top"].major_ticklabels.set_axis_direction("top")
    ax1.axis["top"].label.set_axis_direction("top")

    ax1.axis["left"].label.set_text("R")
    ax1.axis["top"].label.set_text(ur"$\alpha$ [\u00b0]")

    # create a parasite axes
    aux_ax = ax1.get_aux_axes(tr)

    aux_ax.patch = ax1.patch # for aux_ax to have a clip path as in ax
    ax1.patch.zorder=0.9 # but this has a side effect that the patch is
                         # drawn twice, and possibly over some other
                         # artists. So, we decrease the zorder a bit to
                         # prevent this.

    return ax1, aux_ax

#
# call the plot setup to generate subplots
#

fig = plt.figure(figsize=(5, 5))
fig.subplots_adjust(wspace=0.3, left=0.05, right=0.95)
ax1, aux_ax1 = setup_axes(fig, 111, theta=[0, 90], radius=[0, 1])

#
# generate the data to plot
#
theta = np.linspace(0,90) # in degrees

# 
# populate the subplot with the data
#
im = aux_ax1.imshow(gaps,extent = (x0,x1, y0,y1, ), origin = 'lower',
cmap=plt.get_cmap('nipy_spectral'), norm=matplotlib.colors.LogNorm(vmin = 10**-5 ,vmax=2))

plt.show()  

In [ ]:
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt

#-- Generate Data -----------------------------------------
# Using linspace so that the endpoint of 360 is included...
azimuths = np.radians(np.linspace(0, 90, 100))
zeniths = np.arange(0, 70, 10)
r, theta = np.meshgrid(zeniths, azimuths)
values1 = np.random.random((azimuths.size, zeniths.size))

#-- Plot... ------------------------------------------------
fig, ax1 = plt.subplots(1, 1, figsize=(5,5),subplot_kw=dict(projection='polar'))
p1 = ax1.contourf(theta, r, values1, 100)

#-- obtaining the colormap limits
vmin,vmax = p1.get_clim()
#-- Defining a normalised scale
cNorm = mpl.colors.Normalize(vmin=vmin, vmax=vmax)
#-- Creating a new axes at the right side
ax2 = fig.add_axes([0.9, 0.1, 0.03, 0.8])
#-- Plotting the colormap in the created axes
cb1 = mpl.colorbar.ColorbarBase(ax2, norm=cNorm)
fig.subplots_adjust(left=0.05,right=0.85)
plt.show()


In [ ]:
from matplotlib.transforms import Affine2D
import mpl_toolkits.axisartist.floating_axes as floating_axes
import numpy as np
import mpl_toolkits.axisartist.angle_helper as angle_helper
from matplotlib.projections import PolarAxes
from mpl_toolkits.axisartist.grid_finder import (FixedLocator, MaxNLocator,
                                                 DictFormatter)
import matplotlib.pyplot as plt

def setup_axes2(fig, rect):
    """
    this is the axis on the outside
    """
    tr = PolarAxes.PolarTransform()

    pi = np.pi
    angle_ticks = [(0, r"$0$"),
                   (.25*pi, r"$\frac{1}{4}\pi$"),
                   (.5*pi, r"$\frac{1}{2}\pi$")]
    grid_locator1 = FixedLocator([v for v, s in angle_ticks])
    tick_formatter1 = DictFormatter(dict(angle_ticks))

    grid_locator2 = MaxNLocator(2)

    grid_helper = floating_axes.GridHelperCurveLinear(
        tr, extremes=(.5*pi, 0, 2, 1),
        grid_locator1=grid_locator1,
        grid_locator2=grid_locator2,
        tick_formatter1=tick_formatter1,
        tick_formatter2=None,)

    ax1 = floating_axes.FloatingSubplot(fig, rect, grid_helper=grid_helper, )
    fig.add_subplot(ax1)

    # create a parasite axes whose transData in RA, cz
    aux_ax = ax1.get_aux_axes(tr)

    aux_ax.patch = ax1.patch  # for aux_ax to have a clip path as in ax
    ax1.patch.zorder = 0.9  # but this has a side effect that the patch is
    # drawn twice, and possibly over some other
    # artists. So, we decrease the zorder a bit to
    # prevent this.

    return ax1, aux_ax

def setup_axes3(fig, rect):
    """
    This is the axis on the inside
    """
    tr = PolarAxes.PolarTransform()

    pi = np.pi
    angle_ticks = [(0, r"$0$"),
                   (.25*pi, r"$\frac{1}{4}\pi$"),
                   (.5*pi, r"$\frac{1}{2}\pi$")]
    grid_locator1 = FixedLocator([v for v, s in angle_ticks])
    tick_formatter1 = DictFormatter(dict(angle_ticks))

    grid_locator2 = MaxNLocator(2)

    grid_helper = floating_axes.GridHelperCurveLinear(
        tr, extremes=(.5*pi, 0, 1, 0),
        grid_locator1=grid_locator1,
        grid_locator2=grid_locator2,
        tick_formatter1=tick_formatter1,
        tick_formatter2=None,)

    ax1 = floating_axes.FloatingSubplot(fig, rect, grid_helper=grid_helper)
    fig.add_subplot(ax1)

    # create a parasite axes whose transData in RA, cz
    aux_ax = ax1.get_aux_axes(tr)

    aux_ax.patch = ax1.patch  # for aux_ax to have a clip path as in ax
    ax1.patch.zorder = 0.9  # but this has a side effect that the patch is
    # drawn twice, and possibly over some other
    # artists. So, we decrease the zorder a bit to
    # prevent this.

    return ax1, aux_ax

def plot_c(data,fig,rect):
    
    x0, x1, y0, y1 = 0, 1, 0,1 # bounds of data 
    axc = fig.add_subplot(rect)
    
    plt.ylabel(r'$A_{anc}(s)$') 
    plt.xlabel(r'$A_{log}(s)$')

    im = axc.imshow(data,extent = (x0,x1, y0,y1, ), origin = 'lower',
cmap=plt.get_cmap('nipy_spectral'), norm=matplotlib.colors.LogNorm(vmin = 10**-5 ,vmax=2))#vmin=1600, vmax=5000
    return axc
    
    

##########################################################
p_thetas = np.asarray(mylib.load_object(r"explore_from_base_QUBITS=4")['thetas'])
prob = np.asarray(mylib.load_object(r"explore_from_base_QUBITS=4")['last_prob'])
gaps = mylib.load_object('Jupyter013.001')

fig = plt.figure(figsize=(5., 5.))
fig.subplots_adjust(wspace=0., left=0.1, right=.95)
axc = plot_c(gaps,fig,223)
axc.axis('off')
ax2, aux_ax2 = setup_axes2(fig, 111)
theta = np.random.rand(30)*.5*np.pi
radius = np.random.rand(30) + 1.
aux_ax2.plot(p_thetas, prob+1)

ax3, aux_ax3 = setup_axes3(fig, 223)
ax3.set_axis_bgcolor('none')
aux_ax3.set_axis_bgcolor('none')

theta = np.linspace(0,.5*np.pi,10)  # in degrees
radius = 0.5*np.ones(10)#np.linspace(0,1,10)
aux_ax3.plot(theta, radius)


plt.show()


In [ ]:
floating_axes.GridHelperCurveLinear?


In [ ]:
p_thetas = np.asarray(mylib.load_object(r"explore_from_base_QUBITS=4")['thetas'])
prob = np.asarray(mylib.load_object(r"explore_from_base_QUBITS=4")['last_prob'])